In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import micasense.image as image
import micasense.capture as capture
import os, glob
from pathlib import Path

In [ ]:
# Set the base path to your data
imagePath = Path("../data/tabakas_hand")

# Identify the survey (actual) images
# Replace 'IMG_0015_*' with the prefix for your specific survey capture
imageNames = list(imagePath.glob('IMG_0015_*.tif'))
imageNames = [x.as_posix() for x in imageNames]

# Identify the calibration panel images
# Replace 'IMG_0002_*' with the prefix for your panel capture
panelNames = list(imagePath.glob('IMG_0002_*.tif'))
panelNames = [x.as_posix() for x in panelNames]

print(f"Survey images found ({len(imageNames)}):")
for name in imageNames: print(f" - {os.path.basename(name)}")

print(f"\nPanel images found ({len(panelNames)}):")
for name in panelNames: print(f" - {os.path.basename(name)}")

In [ ]:
# Load the survey images into a Capture object
survey_capture = capture.Capture.from_filelist(imageNames)
img_type = "radiance"

# Load the calibration panel images into a Capture object
panel_capture = capture.Capture.from_filelist(panelNames)
#thecapture = capture.Capture.from_filelist(imageNames)
# Verification
print(f"Survey Capture ID: {survey_capture.uuid} ({survey_capture.camera_model})")
print(f"Panel Capture ID: {panel_capture.uuid} ({panel_capture.camera_model})")

##  <p style="color:blue; font-size:18px">Foto uŽkrautos, toliau skaičiuosim atspindžius</p>

# Calculate Irradiance (from Panel)
##  <p style="color:green; font-size:18px">"Irradiance" duomenys iš kalibracinės paneles. VIsų šešių nuotrauko atspindžio energija.</p>

In [ ]:
# Calculate the sunlight intensity (irradiance) using the panel capture
irradiance_list = panel_capture.panel_irradiance()

# Verify the numbers
print(f"Irradiance values (6 bands): {irradiance_list}")

##  <p style="color:green; font-size:18px">Tiriamos nuotraukos kompensacija pgl. kalibracinės panleles atspinėtą energiją.</p>

In [ ]:
# Use the panel numbers to calibrate the survey images
survey_capture.compute_reflectance(irradiance_list)

print("Reflectance computed for all survey bands.")

##  <p style="color:blue; font-size:18px">Patikrinimas. Keleto centrinių, nuotraukos pixelių, atspindžio skaitinės vertės prieš ir po kompensacijos</p>

In [ ]:
import numpy as np

# Select the Red band (index 2)
img = survey_capture.images[2]

# Get a 3x3 patch from the center
radiance_data = img.undistorted_radiance()
reflectance_data = img.reflectance()
cy, cx = radiance_data.shape[0] // 2, radiance_data.shape[1] // 2

print("--- Radiance (Raw Power) ---")
print(radiance_data[cy-1:cy+2, cx-1:cx+2])

print("\n--- Calibrated Reflectance (0-1 Scale) ---")
print(np.round(reflectance_data[cy-1:cy+2, cx-1:cx+2], 4))

##  <p style="color:blue; font-size:18px">Patikrinimas. Visualinis nuotraukų palyginimas</p>

In [ ]:
import matplotlib.pyplot as plt

# 1. Get the Green band
green_band_idx = 1
img = survey_capture.images[green_band_idx]

# 2. Get the two data versions
# 'undistorted_radiance' is the raw power (the 0.29... values)
# 'reflectance' is the calibrated value (the 0.22... values)
before_data = img.undistorted_radiance()
after_data = img.reflectance()

# 3. Plot side-by-side with a FIXED scale (0.0 to 1.0)
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# Before Calibration
im0 = axes[0].imshow(before_data, vmin=0, vmax=1, cmap='gray')
axes[0].set_title(f"Green Band: BEFORE (Radiance)\nValue ~0.29")
plt.colorbar(im0, ax=axes[0], fraction=0.046, pad=0.04)

# After Calibration
im1 = axes[1].imshow(after_data, vmin=0, vmax=1, cmap='gray')
axes[1].set_title(f"Green Band: AFTER (Reflectance)\nValue ~0.22")
plt.colorbar(im1, ax=axes[1], fraction=0.046, pad=0.04)

plt.tight_layout()
plt.show()

#  <p style="color:blue; font-size:18px">Foto perskaičiuotos į atspindžius ir kompensuotos pgl kalibracijos plokštelę</p>

## <p style="color:green; font-size:16px">Pirminės matricų(fotografijų) prasislinkimo vertės dėl kameros konstrucijos, šeši skirtingi lešiai, truputį skirtingos jų vietos, taigi visos nuotraukos iš skirtingo taškos, "prasislinkę"</p>

In [ ]:
# Get initial alignment transformations (warp matrices) from the camera's metadata
warp_matrices = survey_capture.get_warp_matrices()

print(f"Factory warp matrices loaded for {len(warp_matrices)} bands.")

## Nuotraukų sulygiavims pgl. panchromatinę(didžiausią) nuotrauką

### PIrma sustumdom nuotraukas pgl. iš akies nustatytas koordinates, tada praleidžiam matematinį algoritmą SIFT tikslesnem sulygiavimui, jei iškart mėginsim SIFT visų pirma užtruks, antrą jei perstumimai didelis negausim jokio rezultato.

In [ ]:
#veikiantis greitas scriptas su rankiniu budu nustatytais ir ivestais lygiavimo koordinaciu pataisymais

import cv2
import numpy as np
import matplotlib.pyplot as plt

def align_sift(img_target, img_ref, band_idx):
    print(f"Fine-tuning Band {band_idx} with SIFT...")
    ref = cv2.normalize(img_ref, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
    tgt = cv2.normalize(img_target, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
    
    sift = cv2.SIFT_create(nfeatures=2000)
    kp1, des1 = sift.detectAndCompute(ref, None)
    kp2, des2 = sift.detectAndCompute(tgt, None)
    
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)
    good = [m for m, n in matches if m.distance < 0.7 * n.distance]
    
    if len(good) > 15:
        src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
        M, _ = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)
        print(f"  --> Success: {len(good)} matches for Band {band_idx}")
        return M
    print(f"  --> Failed: Using manual only for Band {band_idx}.")
    return np.eye(3)

# 1. Setup
pre_matrices = [np.eye(3, dtype=np.float32) for _ in range(6)]
pre_matrices[1][0,2], pre_matrices[1][1,2] = 70.0, -25.0
pre_matrices[3][0,2], pre_matrices[3][1,2] = 10.0, 40.0
pre_matrices[0][0,2], pre_matrices[0][1,2] = -70.0, 80.0
pre_matrices[2][0,2], pre_matrices[2][1,2] = 70.0, -40.0
pre_matrices[4][0,2], pre_matrices[4][1,2] = 10.0, -40.0

final_aligned = [None] * 6
manual_only = [None] * 6  # DEFINED HERE
ref_img = survey_capture.images[5].undistorted_radiance()
h, w = ref_img.shape

# 2. Process
for i in range(6):
    img = cv2.resize(survey_capture.images[i].undistorted_radiance(), (w, h))
    
    # Apply manual shift and store it
    pre_warped = cv2.warpPerspective(img, pre_matrices[i], (w, h), flags=cv2.WARP_INVERSE_MAP)
    manual_only[i] = pre_warped
    
    if i == 5:
        final_aligned[5] = ref_img
        continue
        
    m_fine = align_sift(pre_warped, ref_img, i)
    m_total = m_fine @ np.linalg.inv(pre_matrices[i])
    final_aligned[i] = cv2.warpPerspective(img, m_total, (w, h), flags=cv2.INTER_LANCZOS4)

# 3. Plot Comparison
plt.figure(figsize=(15, 7))

# Left: Manual
manual_rgb = np.stack([manual_only[2], manual_only[1], manual_only[0]], axis=2)
m_disp = np.clip(manual_rgb / np.percentile(manual_rgb, 98), 0, 1)
plt.subplot(1, 2, 1)
plt.imshow(m_disp)
plt.title("Before SIFT (Manual Only)")
plt.axis('off')

# Right: Final
final_rgb = np.stack([final_aligned[2], final_aligned[1], final_aligned[0]], axis=2)
f_disp = np.clip(final_rgb / np.percentile(final_rgb, 98), 0, 1)
plt.subplot(1, 2, 2)
plt.imshow(f_disp)
plt.title("After SIFT (Fine-tuned)")
plt.axis('off')

plt.tight_layout()
plt.show()

plt.imsave("Before_SIFT_Manual_Only.png", m_disp)
plt.imsave("After_SIFT_Fine_tuned.png", f_disp)

print("Images saved successfully.")

In [ ]:
# 1. Manually set the rig relatives in the capture object
# This replaces the factory settings with your research-optimized ones
for i in range(5):
    survey_capture.images[i].meta.set_calibration_rig_relatives(pre_matrices[i])

# 2. Now run SIFT
# It will now use your pre_matrices as the baseline for 'warp_matrices_calibrated'
warp_matrices_SIFT = survey_capture.SIFT_align_capture(ref=5)

In [ ]:
import micasense.imageutils as imageutils
import time

st = time.time()
# 1. Use your manual matrices as the starting point
# (Assuming you still have 'pre_matrices' from our previous steps)
initial_matrices = [m.astype(np.float32) for m in pre_matrices]

# 2. Run SIFT, but tell it to START from your manual alignment
# This helps SIFT not get "lost" in the background
#warp_matrices_SIFT = survey_capture.SIFT_align_capture(
#    ref=5, 
#    min_matches=20,     # Increased for better accuracy
#    err_red=200.0,      # Tightened error threshold
#    err_blue=200.0,
#    verbose=1
#)

warp_matrices_SIFT = survey_capture.SIFT_align_capture(
    ref=5, 
    min_matches=20,
    err_red=200.0,
    err_blue=200.0,
    verbose=1
)


# 3. Apply the sharpening using the refined matrices
sharpened_stack, upsampled = survey_capture.radiometric_pan_sharpened_aligned_capture(
    warp_matrices=warp_matrices_SIFT, 
    irradiance_list=irradiance_list, 
    img_type='radiance'
)

print("Pansharpened shape:", sharpened_stack.shape)
print("Upsampled shape:", upsampled.shape)
# re-assign to im_aligned to match rest of code 
im_aligned = upsampled
et = time.time()
elapsed_time = et - st
print('Alignment and pan-sharpening time:', int(elapsed_time), 'seconds')

# 4. Extract RGB from upsampled (Indices 2, 1, 0) and Normalize
rgb_sharp = np.stack([upsampled[:,:,2], upsampled[:,:,1], upsampled[:,:,0]], axis=2)

# High-quality stretch for saving
rgb_sharp_disp = np.clip(rgb_sharp / np.percentile(rgb_sharp, 98), 0, 1)

# 5. Save the file
plt.imsave("micasense_alignment.png", rgb_sharp_disp)

print("Micasense pan-sharpened result saved as micasense_alignment.png")

In [ ]:
# figsize=(30,23) # use this size for full-image-resolution display
figsize=(16,13)   # use this size for export-sized display

rgb_band_indices = [survey_capture.band_names_lower().index('red'),
                    survey_capture.band_names_lower().index('green'),
                    survey_capture.band_names_lower().index('blue')]
cir_band_indices = [survey_capture.band_names_lower().index('nir'),
                    survey_capture.band_names_lower().index('red'),
                    survey_capture.band_names_lower().index('green')]

# Create normalized stacks for viewing
im_display = np.zeros((im_aligned.shape[0],im_aligned.shape[1],im_aligned.shape[2]), dtype=np.float32)
im_min = np.percentile(im_aligned[:,:,rgb_band_indices].flatten(), 0.5)  # modify these percentiles to adjust contrast
im_max = np.percentile(im_aligned[:,:,rgb_band_indices].flatten(), 99.5)  # for many images, 0.5 and 99.5 are good values


im_display_sharp = np.zeros((sharpened_stack.shape[0],sharpened_stack.shape[1],sharpened_stack.shape[2]), dtype=np.float32 )
im_min_sharp = np.percentile(sharpened_stack[:,:,rgb_band_indices].flatten(), 0.5)  # modify these percentiles to adjust contrast
im_max_sharp = np.percentile(sharpened_stack[:,:,rgb_band_indices].flatten(), 99.5)  # for many images, 0.5 and 99.5 are good values


# for rgb true color, we use the same min and max scaling across the 3 bands to 
# maintain the "white balance" of the calibrated image
for i in rgb_band_indices:
    im_display[:,:,i] =  imageutils.normalize(im_aligned[:,:,i], im_min, im_max)

    im_display_sharp[:,:,i] = imageutils.normalize(sharpened_stack[:,:,i], im_min_sharp, im_max_sharp)

rgb = im_display[:,:,rgb_band_indices]


rgb_sharp = im_display_sharp[:,:,rgb_band_indices]

nir_band = survey_capture.band_names_lower().index('nir')
red_band = survey_capture.band_names_lower().index('red')

ndvi = (im_aligned[:,:,nir_band] - im_aligned[:,:,red_band]) / (im_aligned[:,:,nir_band] + im_aligned[:,:,red_band])

# for cir false color imagery, we normalize the NIR,R,G bands within themselves, which provides
# the classical CIR rendering where plants are red and soil takes on a blue tint
for i in cir_band_indices:
    im_display[:,:,i] =  imageutils.normalize(im_aligned[:,:,i])

cir = im_display[:,:,cir_band_indices]

fig, (ax1,ax2) = plt.subplots(1, 2, figsize=figsize)

    #fig, ax1 = plt.subplots(1, 1, figsize=figsize)
ax1.set_title("Red-Green-Blue Composite")
ax1.imshow(rgb)

ax2.set_title("Red-Green-Blue Composite (pan-sharpened)")
ax2.imshow(rgb_sharp)

fig, (ax3,ax4) = plt.subplots(1, 2, figsize=figsize)
ax3.set_title("NDVI")
ax3.imshow(ndvi)
ax4.set_title("Color Infrared (CIR) Composite")
ax4.imshow(cir)

# set custom lims if you want to zoom in to image to see more detail 
# this is useful for comparing upsampled and pan-sharpened stacks 
# custom_xlim=(1500,2000)
# custom_ylim=(2000,1500)
# plt.setp([ax1,ax2,ax3,ax4], xlim=custom_xlim, ylim=custom_ylim)

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Micasense upsampled order is typically [B, G, R, RE, NIR]
# We want R(2), G(1), B(0)
rgb = np.stack([upsampled[:,:,2], upsampled[:,:,1], upsampled[:,:,0]], axis=2)

# Normalize for display
rgb_display = np.clip(rgb / np.percentile(rgb, 98), 0, 1)

plt.figure(figsize=(10, 10))
plt.imshow(rgb_display)
plt.title("Pan-Sharpened RGB Composite")
plt.axis('off')
plt.show()

In [ ]:
# NIR is index 4, Red is index 2
nir = upsampled[:,:,4]
red = upsampled[:,:,2]

with np.errstate(divide='ignore', invalid='ignore'):
    ndvi_sharp = (nir - red) / (nir + red)

plt.figure(figsize=(10, 10))
plt.imshow(ndvi_sharp, cmap='RdYlGn', vmin=-1, vmax=1)
plt.colorbar(label='NDVI')
plt.title("High-Res Pan-Sharpened NDVI")
plt.axis('off')
plt.show()

In [ ]:
# 3. Enhanced "Natural" RGB Plot
rgb = np.stack([final_aligned[2], final_aligned[1], final_aligned[0]], axis=2)

# A. Histogram Stretch (Individual channel normalization)
for i in range(3):
    low, high = np.percentile(rgb[:,:,i], (2, 99)) # Ignore bottom 2% and top 1% noise
    rgb[:,:,i] = np.clip((rgb[:,:,i] - low) / (high - low), 0, 1)

# B. Apply Gamma Correction (typically 0.45 to 0.67 makes it look natural)
gamma = 0.6
rgb_natural = np.power(rgb, gamma)

plt.figure(figsize=(12, 12))
plt.imshow(rgb_natural)
plt.title("Natural Look: Histogram Stretch + Gamma Correction")
plt.axis('off')
plt.show()